In [1]:
import mysql.connector as mysql
import pandas as pd
import numpy as np

In [2]:
connection = mysql.connect(host='localhost', user='root', password='', database='we bake')

In [3]:
query = "SELECT * FROM productitem WHERE availability=1"
df = pd.read_sql(query, connection)
df.head()

C:\Users\anuda\AppData\Local\Temp\ipykernel_13908\1296036662.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


,itemid,retailprice,cost,itemdescription,itemname,unit,category,Itemcode,imagelink,availability,ipc
0,53,170,110,Normal bread 450g,Bread(Normal),,Bread,BR00001,BR00001.jpeg,1,5
1,54,200,160,Sliced sandwich Bread,Sandwich Bread,,Bread,BR00054,BR00054.jpeg,1,5
2,55,130,100,350g normal bread,Bread(Thin),,Bread,BR00055,BR00055.jpeg,1,2
3,56,120,90,Fish Pastrie,Fish Pastrie,,Pastries,PA00056,PA00056.jpeg,1,5
4,57,150,100,Pastries Chicken,Chicken Pastries,,Pastries,PA00057,PA00057.jpeg,1,5


In [4]:
df = df[['itemid', 'itemdescription', 'itemname', 'category']]
df.head()

,itemid,itemdescription,itemname,category
0,53,Normal bread 450g,Bread(Normal),Bread
1,54,Sliced sandwich Bread,Sandwich Bread,Bread
2,55,350g normal bread,Bread(Thin),Bread
3,56,Fish Pastrie,Fish Pastrie,Pastries
4,57,Pastries Chicken,Chicken Pastries,Pastries


In [5]:
df["index"] = df.index 

In [6]:
df.shape

(30, 5)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

def combine_features(row):
    return row['itemdescription'] + ' ' + row['itemname'] + ' ' + row['category']

In [8]:
df['combined_features'] = df.apply(combine_features, axis=1)

In [9]:
from sklearn.metrics.pairwise import cosine_similarity


count_matrix = cv.fit_transform(df['combined_features'])

cosine_sim = cosine_similarity(count_matrix)

In [10]:
type(cosine_sim)

numpy.ndarray

In [11]:
cosine_sim.shape

(30, 30)

In [12]:
def get_name_from_index(index):
    return df[df.index	 == index]["itemname"].values[0]

def get_index_from_name(itemname):
    return df[df.itemname == itemname]["index"].values[0]

In [13]:
product_user_likes = "Bread(Normal)"
product_index = get_index_from_name(product_user_likes)
product_index

0

In [14]:
product_name = get_name_from_index(0)
print(product_name)

Bread(Normal)


In [15]:
similar_products = list(enumerate(cosine_sim[product_index]))

In [16]:
sorted_similar_product = sorted(similar_products,key=lambda x:x[1],reverse=True)[1:]

i=0
print("Top 5 similar prdocut to "+product_user_likes+" are:\n")
for element in similar_products:
    element = get_name_from_index(element[0])
    if element != product_user_likes:
        print(i+1,element)
        i=i+1
        if i>4:
            break

Top 5 similar prdocut to Bread(Normal) are:

1 Sandwich Bread
2 Bread(Thin)
3 Fish Pastrie
4 Chicken Pastries
5 Vegi Pastries


In [17]:
print(cosine_sim)

[[1.         0.64285714 0.84866842 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.64285714 1.         0.69436507 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.84866842 0.69436507 1.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         1.         0.2773501  0.30151134
  0. 

In [18]:
with open('output.txt', 'wb') as outfile:
    np.save(outfile, cosine_sim)

In [19]:
cosine_sim_loaded = np.load('output.txt')
print(cosine_sim_loaded)

[[1.         0.64285714 0.84866842 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.64285714 1.         0.69436507 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.84866842 0.69436507 1.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         1.         0.2773501  0.30151134
  0. 

In [20]:
similar_products_loaded = list(enumerate(cosine_sim_loaded[product_index]))

sorted_similar_product = sorted(similar_products_loaded,key=lambda x:x[1],reverse=True)[1:]

i=0
print("Top 5 similar prdocut to "+product_user_likes+" are:\n")
for element in similar_products:
    element = get_name_from_index(element[0])
    if element != product_user_likes:
        print(i+1,element)
        i=i+1
        if i>4:
            break

Top 5 similar prdocut to Bread(Normal) are:

1 Sandwich Bread
2 Bread(Thin)
3 Fish Pastrie
4 Chicken Pastries
5 Vegi Pastries
